RESUMEN/PREAMBULO

Primero importaremos la base de datos del IPC del INE.

In [20]:
import pandas as pd
url='https://www.ine.cl/docs/default-source/%C3%ADndice-de-precios-al-consumidor/cuadros-estadisticos/base-2018/series-de-tiempo/ipc-xls.xlsx'

df_INE=pd.read_excel(url,skiprows=3)

Ahora filtramos solamente productos, ya que estos son el nivel más desagregado que publica el INE, y creamos un ID único a cada producto, la ventaja de trabajar con un ID numérico sobre un nombre (por ejemplo: "Paquete Turístico") es que evitamos problemas asociados a la Tipografía (espacios, tildes, mayúsculas, etc). además, cambiaremos la base de tipo "long" a "wide" con la función pivot_table, que es análoga a su contraparte tabla dinámica de MS Excel.

In [21]:
filtro_productos=df_INE['Producto'].astype(str).str.isnumeric()

df_productos=df_INE[filtro_productos]
df_productos['Fecha']=df_productos['Año']*100+df_productos['Mes']
df_productos['ID']=(
df_productos['División']*1e5+
df_productos['Grupo']*1e4+
df_productos['Clase']*1e3+
df_productos['Subclase']*1e2+
df_productos['Producto']*1e0
).astype(int)

df_productos_mensuales=df_productos.pivot_table(values='Índice',index='ID',columns='Fecha')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


Por otro lado, rescataremos los ponderadores de cada producto, para esto fijaremos arbitrariamente un mes en específico y tomaremos esta información, ya que actualmente está duplicada por cada nuevo mes que se agrega en la base.

In [22]:
filtro_mes=df_productos['Fecha']==201901

df_auxiliar_ponderadores=df_productos[['Fecha','Ponderación','ID']][filtro_mes]
df_auxiliar_ponderadores.index=df_auxiliar_ponderadores['ID']
ponderadores=df_auxiliar_ponderadores.sort_index()['Ponderación']

Con la base en formato wide y los ponderadores podemos crear una función que tome estos dos inputs y calcule el nivel del índice. Luego a partir de estos niveles podemos concentrarnos en variaciones (mensuales, interanuales o de otra naturaleza).

In [23]:
def calcula_indice(dataframe,ponderadores):
    numerador=dataframe.mul(ponderadores.values,axis=0).sum()
    denominador=ponderadores.sum()
    niveles=numerador/denominador
    return niveles

niveles_IPC=calcula_indice(df_productos_mensuales,ponderadores)
niveles_IPC.pct_change(1)[1:]

Fecha
201902    0.000462
201903    0.004743
201904    0.002606
201905    0.006044
201906    0.000488
201907    0.002236
201908    0.001872
201909    0.000121
201910    0.008112
201911    0.000848
201912    0.000993
202001    0.005662
202002    0.004484
202003    0.003306
202004   -0.000407
202005   -0.000497
202006   -0.000719
202007    0.001003
202008    0.001321
202009    0.006352
202010    0.006867
202011   -0.001372
202012    0.003379
202101    0.007023
202102    0.001853
dtype: float64

In [24]:
niveles_IPC.pct_change(12)[12:]

Fecha
202001    0.034688
202002    0.038848
202003    0.037362
202004    0.034245
202005    0.027520
202006    0.026280
202007    0.025018
202008    0.024454
202009    0.030837
202010    0.029565
202011    0.027280
202012    0.029729
202101    0.031122
202102    0.028422
dtype: float64

De la misma manera que replicamos el índice General del IPC, podemos replicar sus índices analíticos cambiando el input de ponderadores, por ejemplo, replicando el índice Sin Alimentos ni Energía (IPC SAE).

In [25]:
url_ponderadores='https://www.ine.cl/docs/default-source/%C3%ADndice-de-precios-al-consumidor/metodologias/base-anual-2018-100/canasta-anal%C3%ADticos-ipc-2018=100.xlsx'

df_ponderadores=pd.read_excel(url_ponderadores,skiprows=1)
df_ponderadores['ID']=(
df_ponderadores['D']*1e5+
df_ponderadores['G']*1e4+
df_ponderadores['C']*1e3+
df_ponderadores['SC']*1e2+
df_ponderadores['P']*1e0
).astype(int)

df_ponderadores.sort_values('ID',inplace=True)

In [26]:
filtro_SAE=df_ponderadores['IPC menos Alimentos y Energía']
ponderadores_SAE=ponderadores*filtro_SAE.values
niveles_IPC_SAE=calcula_indice(df_productos_mensuales,ponderadores_SAE)
niveles_IPC_SAE.pct_change(1)[1:]

Fecha
201902    0.002624
201903    0.004343
201904    0.000915
201905    0.003556
201906   -0.000137
201907    0.002970
201908    0.000677
201909    0.000541
201910    0.005876
201911   -0.001557
201912    0.001242
202001    0.003955
202002    0.004265
202003    0.002598
202004   -0.000948
202005    0.000940
202006   -0.000815
202007    0.001061
202008    0.002177
202009    0.003415
202010    0.007322
202011   -0.001959
202012    0.003604
202101    0.004536
202102    0.001947
dtype: float64

In [27]:
niveles_IPC_SAE.pct_change(12)[12:]

Fecha
202001    0.025267
202002    0.026945
202003    0.025160
202004    0.023252
202005    0.020585
202006    0.019894
202007    0.017952
202008    0.019478
202009    0.022406
202010    0.023876
202011    0.023464
202012    0.025878
202101    0.026472
202102    0.024103
dtype: float64

Hasta ahora solo hemos replicado el cálculo de productos a índice general del INE, una posible acción seria a continuación cambiar el input de la base y agregar supuestos sobre el comportamiento futuro de los productos del índice para proyectar que pasara con el índice general, que es lo que hacemos en Pacifico Research.

Otro ejercicio sería modificar los ponderadores y crear un índice personalizado, dependiendo del análisis que se desee realizar. En el último tiempo por motivos de la pandemia una serie de servicios sufrió abruptamente su oferta, siendo los más destacados para motivos del análisis de precios "Servicio de transporte aéreo" y "Paquete Turístico". El INE opto metodológicamente por imputar estos productos por el método de arrastre (mantener el ultimo valor recogido, en este caso abril del 2020). ¿Qué hubiese pasado hipotéticamente si el INE hubiese removido estos productos del indicador, en vez de imputarlos?

In [28]:
ponderadores_personalizado=ponderadores.copy()
ponderadores_personalizado[732101]=0  #"Servicio de transporte aéreo"
ponderadores_personalizado[951101]=0 #"Paquete Turístico"

In [29]:
niveles_IPC_personalizado=calcula_indice(df_productos_mensuales,ponderadores_personalizado)
niveles_IPC_personalizado.pct_change(1)[1:]

Fecha
201902    0.000947
201903    0.004882
201904    0.002601
201905    0.003961
201906    0.000489
201907    0.002151
201908    0.000857
201909    0.001638
201910    0.006286
201911    0.000636
201912    0.001215
202001    0.005042
202002    0.005094
202003    0.004285
202004   -0.000031
202005   -0.000505
202006   -0.000731
202007    0.001020
202008    0.001342
202009    0.006455
202010    0.006977
202011   -0.001394
202012    0.003433
202101    0.007134
202102    0.001882
dtype: float64

In [30]:
niveles_IPC_personalizado.pct_change(12)[12:]

Fecha
202001    0.031118
202002    0.035390
202003    0.034776
202004    0.032059
202005    0.027467
202006    0.026214
202007    0.025056
202008    0.025553
202009    0.030485
202010    0.031193
202011    0.029101
202012    0.031381
202101    0.033527
202102    0.030225
dtype: float64

In [31]:
niveles_IPC_personalizado.pct_change(12).iloc[-1]-niveles_IPC.pct_change(12).iloc[-1]

0.0018030899850982607

0.2 décimas más hubiese sido el IPC interanual a febrero si los esos 2 productos hubiesen sido removidos en vez de imputados por arrastre. ¿Qué hubiese pasado si en vez de una imputación por arrastre se hubiese usado otro tipo de metodología? Esas y otras preguntas quedan al lector (hint: pueden responderse fácilmente cambiando los inputs respectivos).